<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/TAPT_seq_bn_pretrain_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This is 2 epoch test of the pretraining of the TAPT for amazon helpfulness






# Setup

In [1]:
from time import time
start_time = time()

In [2]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [3]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

In [4]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 915.4 kB/s eta 0:00:00


In [5]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [6]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [7]:
from datasets import load_dataset

pretrain_dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=["test","dev"])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

Generating train split:   0%|          | 0/44876 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9754 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1980 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9754
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1980
})


In [8]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# THIS MUST BE SET TO MATCH  pretraining_adapter_type
from adapters import SeqBnConfig

pretraining_adapter_config = SeqBnConfig()
pretrained_adapter_name = "tapt_seq_bn"
pretrained_adapter_hub_name = f"pretrain_{pretrained_adapter_name}_adpater"

In [10]:
print(pretraining_adapter_config.__class__.__name__)
print(pretrained_adapter_name)

SeqBnConfig
tapt_seq_bn


In [11]:
# Add a new adapter
pretrain_model.add_adapter(pretrained_adapter_name, config=pretraining_adapter_config)

# Add head for masked language modeling
pretrain_model.add_masked_lm_head(pretrained_adapter_name)

# Set the adapter to be used for training
pretrain_model.train_adapter(pretrained_adapter_name)

print(f"Pretrained Adapter Name: {pretrained_adapter_name}")

Pretrained Adapter Name: tapt_seq_bn


In [12]:
pretrain_model.active_adapters

Stack[tapt_seq_bn]

In [13]:
summary = pretrain_model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
tapt_seq_bn              bottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [16]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir="tapt_helpfulness_pretraining_model",
    learning_rate=0.0005, # Paper for DAPT training
    per_device_train_batch_size=32, # 346,m - TRYING SMALLER BATCH SIZE
    per_device_eval_batch_size=32, #346,m - TRYING SMALLER BATCH SIZE
    gradient_accumulation_steps=11, # Creates effective batch of 32*11*6 = 2112
    num_train_epochs=2, # Just one epoch for one pass on data
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="steps",
    eval_steps=500, # Evaluates every 50 steps
    save_strategy="steps",
    save_steps=500, # Checkpoint saves every 500 steps / every 1,000,000 reviews ( 500 * 10 * 34 * 6)
    save_total_limit=2, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
)

In [17]:
from adapters import AdapterTrainer

pretrain_trainer = AdapterTrainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [18]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=54, training_loss=5.99551448115596, metrics={'train_runtime': 420.2118, 'train_samples_per_second': 46.424, 'train_steps_per_second': 0.129, 'total_flos': 5126762083774464.0, 'train_loss': 5.99551448115596, 'epoch': 1.95})


In [19]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print(after_pretrain_eval_results)

After pretraining loss: 3.2234
{'eval_loss': 3.2233963012695312, 'eval_runtime': 21.5303, 'eval_samples_per_second': 91.963, 'eval_steps_per_second': 2.88, 'epoch': 1.95}


In [20]:
pretrain_model.push_adapter_to_hub(
    pretrained_adapter_hub_name,
    pretrained_adapter_name,
    datasets_tag=pretrain_dataset_name
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/pretrain_tapt_seq_bn_adpater/commit/08412a20aa22ff437ad3e0caaad97e12b5b58c1a', commit_message='Upload model', commit_description='', oid='08412a20aa22ff437ad3e0caaad97e12b5b58c1a', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time:.0f} seconds")

Total Time: 580 seconds


# Classification

In [22]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
